In [89]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [90]:
file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'
path_to_end_folder = 'data/'

In [91]:
wb = load_workbook(file_transfer_reestr) # Загружаем книгу

In [92]:
ws = wb.active # получаем рабочий лист

In [93]:
# Получаем датафрейм
df = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=[4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,
                                                           29,32])

In [94]:
df

,5,6,7,8,9,10,11,13,14,15,...,22,23,24,25,26,27,28,29,30,33
0,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",Обозначение части лесотаксационного выдела (ле...,широта,долгота,Год лесоустройства,...,Дата проведения наблюдения (обследования) или ...,"Группа причин повреждения насаждения, код",Повреждаемая (-мые) порода (-ды),"Общий отпад насаждения (усыхания) по запасу, %",Степень поврежденния (поражения) насаждения,Дата внесения данных в реестр,Отметка об актуальности данных,Отметка о необходимости включения в реестр МЗЛ,1,площадь лесотаксационного выдела или его части
1,Улан-Удэнское,Березовское,NaN,39,3,3.1,NaN,NaN,NaN,0,...,2022-02-04 00:00:00,0,0,0,0,0,0,0,2,0
2,Улан-Удэнское,Березовское,NaN,26,11,5.1,NaN,NaN,NaN,0,...,2022-02-04 00:00:00,0,0,0,0,0,0,0,2,0
3,Улан-Удэнское,Березовское,NaN,27,3,4.2,NaN,NaN,NaN,0,...,2022-02-04 00:00:00,0,0,0,0,0,0,0,2,0
4,Улан-Удэнское,Березовское,NaN,39,14,9.6,NaN,NaN,NaN,0,...,2022-02-04 00:00:00,1,СО,более 40%,более 40%,2022-04-05 00:00:00,1,0,1,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77624,Верхне-Баргузинское,Куллукское,,295,4,14.7,NaN,"54,3956013888692","110,845059521438",0,...,2022-05-17 00:00:00,1,0,0,0,0,0,0,2,0
77625,Верхне-Баргузинское,Куллукское,,295,4,14.7,NaN,"54,3967812147783","110,844984328778",0,...,2022-05-17 00:00:00,1,0,0,0,0,0,0,2,0
77626,Верхне-Баргузинское,Куллукское,,295,4,14.7,NaN,"54,3942930284415","110,846249502594",0,...,2022-05-17 00:00:00,1,0,0,0,0,0,0,2,0
77627,Верхне-Баргузинское,Куллукское,,295,4,14.7,NaN,"54,3966078166225","110,842719272326",0,...,2022-05-17 00:00:00,1,0,0,0,0,0,0,2,0


In [95]:
# Фильтруем датафрейм
transfer_df=df[df[30] == 1]


In [96]:
transfer_df.columns

Int64Index([ 5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23,
            24, 25, 26, 27, 28, 29, 30, 33],
           dtype='int64')

In [97]:
transfer_df.shape

(58811, 25)

In [98]:
# перемещаем площадь выдела
transfer_df.insert(7,12,transfer_df[33])

In [99]:
# удаляем лишний столбец с площадью выдела и признаков внесения в реестр
transfer_df.drop(columns=[30,33],inplace=True)


In [100]:
transfer_df

,5,6,7,8,9,10,11,12,13,14,...,19,20,22,23,24,25,26,27,28,29
0,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",Обозначение части лесотаксационного выдела (ле...,площадь лесотаксационного выдела или его части,широта,долгота,...,Отметка об аренде,"Источник данных, код",Дата проведения наблюдения (обследования) или ...,"Группа причин повреждения насаждения, код",Повреждаемая (-мые) порода (-ды),"Общий отпад насаждения (усыхания) по запасу, %",Степень поврежденния (поражения) насаждения,Дата внесения данных в реестр,Отметка об актуальности данных,Отметка о необходимости включения в реестр МЗЛ
4,Улан-Удэнское,Березовское,NaN,39,14,9.6,NaN,2.3,NaN,NaN,...,0,21,2022-02-04 00:00:00,1,СО,более 40%,более 40%,2022-04-05 00:00:00,1,0
5,Улан-Удэнское,Березовское,NaN,39,8,2,NaN,0.8,NaN,NaN,...,0,21,2022-02-04 00:00:00,1,СО,более 40%,более 40%,2022-04-05 00:00:00,1,0
6,Улан-Удэнское,Березовское,NaN,39,9,10.5,NaN,0.1,NaN,NaN,...,0,21,2022-02-04 00:00:00,1,СО,более 40%,более 40%,2022-04-05 00:00:00,1,0
7,Улан-Удэнское,Березовское,NaN,27,21,2.2,NaN,0.2,NaN,NaN,...,0,21,2022-02-04 00:00:00,1,Б,более 40%,более 40%,2022-04-05 00:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77604,Верхне-Баргузинское,Куллукское,,278,16,1.4,NaN,1.4,"54,424793412566","110,855589602686",...,0,23,2022-05-17 00:00:00,1,0,"4,1-10%","4,1-10%",2022-08-31 00:00:00,1,0
77611,Верхне-Баргузинское,Куллукское,,282,1,2.9,NaN,0.8,"54,4152337244119","110,755870037421",...,0,23,2022-05-17 00:00:00,1,0,"4,1-10%","4,1-10%",2022-08-31 00:00:00,1,0
77612,Верхне-Баргузинское,Куллукское,,282,1,2.9,NaN,0.5,"54,4172841007572","110,755748033173",...,0,23,2022-05-17 00:00:00,1,0,"4,1-10%","4,1-10%",2022-08-31 00:00:00,1,0
77613,Верхне-Баргузинское,Куллукское,,282,6,4.2,NaN,4.4,"54,414440295182","110,75700823851",...,0,23,2022-05-17 00:00:00,1,0,"4,1-10%","4,1-10%",2022-08-31 00:00:00,1,0


In [101]:
# Присваиваем пустое значение для колонки 21 исполнитель, чтобы

In [102]:
transfer_df.columns

Int64Index([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22,
            23, 24, 25, 26, 27, 28, 29],
           dtype='int64')

In [103]:
out_lst = transfer_df.values.tolist()

In [104]:
# Добавляем данные в файл
for row in out_lst:
    ws.append(row)

In [105]:
# Сохраняем
t = time.localtime()
current_time = time.strftime('%H_%M_%S %d.%m.%Y', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder}/Реестр УПП с добавлением {current_time}.xlsx')